In [1]:
import pandas as pd

In [5]:
Trip_Staging_read_df = pd.read_csv('CLIWOC15.csv')

/var/folders/6c/xgblq5v97lb39fths3r32xvm0000gn/T/ipykernel_94173/431724778.py:1: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  Trip_Staging_read_df = pd.read_csv('CLIWOC15.csv')


In [7]:
# Filter needed columns for Trips_Staging
TripStaging_df = Trip_Staging_read_df.filter(['ShipName','ShipType','Nationality', 'RecID', 'Year',
                                             'Month', 'Day', 'Distance', 'ShipSpeed', 'drLatDeg'])
TripStaging_df

,ShipName,ShipType,Nationality,RecID,Year,Month,Day,Distance,ShipSpeed,drLatDeg
0,LA ORBE,FRAGATA,Spanish,108,1800,5,2,NaN,NaN,NaN
1,SAN CARLOS,PAQUEBOTE,Spanish,109,1790,4,11,NaN,NaN,NaN
2,SAN CARLOS,PAQUEBOTE,Spanish,110,1790,4,12,NaN,NaN,NaN
3,SAN CARLOS,PAQUEBOTE,Spanish,111,1790,4,13,NaN,NaN,NaN
4,SAN CARLOS,PAQUEBOTE,Spanish,112,1790,4,14,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
280275,Euryalus,5th Rate,British,280276,1805,2,4,84.0,NaN,42.0
280276,Euryalus,5th Rate,British,280277,1805,2,5,107.0,NaN,41.0
280277,Euryalus,5th Rate,British,280278,1805,2,6,189.0,NaN,38.0
280278,Euryalus,5th Rate,British,280279,1805,2,7,70.0,NaN,37.0


In [8]:
TripStaging_df.dtypes

ShipName        object
ShipType        object
Nationality     object
RecID            int64
Year             int64
Month            int64
Day              int64
Distance       float64
ShipSpeed      float64
drLatDeg       float64
dtype: object

In [9]:
TripStaging_df.count()

ShipName       280280
ShipType       161537
Nationality    280280
RecID          280280
Year           280280
Month          280280
Day            280280
Distance       168970
ShipSpeed          37
drLatDeg       155413
dtype: int64

In [10]:
import psycopg2 as pg

conn = pg.connect(user = "postgres",
                 password = "BU669",
                 host = "127.0.0.1",
                 port = "5432",
                 database = "Assignment_3B")
cur = conn.cursor()

In [11]:
# Create Trips_Staging in database
cur.execute("""CREATE TABLE IF NOT EXISTS Trips_Staging(
                ShipName varchar ,ShipType varchar, Nationality varchar, 
                RecID int, Year int, Month int, Day int, 
                Distance Float, ShipSpeed Float, drLatDeg Int);""")
conn.commit()

In [12]:
#Load data into DBMS
for index, row in TripStaging_df.iterrows():
    ShipName = row.ShipName if not pd.isna(row.ShipName) else None
    ShipType = row.ShipType if not pd.isna(row.ShipType) else None
    Nationality = row.Nationality if not pd.isna(row.Nationality) else None
    Distance = row.Distance if not pd.isna(row.Distance) else None
    ShipSpeed = row.ShipSpeed if not pd.isna(row.ShipSpeed) else None
    drLatdeg = row.drLatDeg if not pd.isna(row.drLatDeg) else None
    cur.execute("""INSERT INTO Trips_Staging (ShipName, ShipType, Nationality, 
                                        RecID, Year, Month, Day,
                                        Distance, ShipSpeed, drLatDeg) 
                                        values (%s,%s,%s,%s,%s,%s,%s,%s,%s, %s)""",
                (ShipName, ShipType, Nationality, row.RecID, row.Year, 
                 row.Month, row.Day, Distance, ShipSpeed, drLatdeg))
conn.commit()

In [38]:
#Extra Credit
cur.execute("""CREATE TABLE IF NOT EXISTS Trips_Staging_weather(
                ShipName varchar ,ShipType varchar, Nationality varchar, 
                RecID int, Year int, Month int, Day int, 
                Distance Float, ShipSpeed Float, drLatDeg Int,
                Weather Varchar, WindForce Varchar, WindDirection Varchar, PrecipitationDescriptor varchar);""")
conn.commit()

In [46]:
TripStaging_weather_df = Trip_Staging_read_df.filter(['ShipName','ShipType','Nationality', 'RecID', 'Year',
                                             'Month', 'Day', 'Distance', 'ShipSpeed', 'drLatDeg', 
                                                'Weather', 'WindForce', 'WindDirection', 'PrecipitationDescriptor'])

In [53]:
for index, row in TripStaging_weather_df.iterrows():
    ShipName = row.ShipName if not pd.isna(row.ShipName) else None
    ShipType = row.ShipType if not pd.isna(row.ShipType) else None
    Nationality = row.Nationality if not pd.isna(row.Nationality) else None
    Distance = row.Distance if not pd.isna(row.Distance) else None
    ShipSpeed = row.ShipSpeed if not pd.isna(row.ShipSpeed) else None
    drLatdeg = row.drLatDeg if not pd.isna(row.drLatDeg) else None
    Weather = row.Weather if not pd.isna(row.Weather) else None
    WindForce = row.WindForce if not pd.isna(row.WindForce) else None
    WindDirection = row.WindDirection if not pd.isna(row.WindDirection) else None
    PrecipitationDescriptor = row.PrecipitationDescriptor if not pd.isna(row.PrecipitationDescriptor) else None
    cur.execute("""INSERT INTO Trips_Staging_weather (ShipName, ShipType, Nationality, 
                                        RecID, Year, Month, Day,
                                        Distance, ShipSpeed, drLatDeg,
                                        Weather, WindForce, WindDirection, PrecipitationDescriptor) 
                                        values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s, %s, %s, %s)""",
(ShipName, ShipType, Nationality, row.RecID, row.Year,row.Month, row.Day, Distance, ShipSpeed, drLatdeg, Weather, WindForce, 
                 WindDirection, PrecipitationDescriptor))
conn.commit()


In [49]:
TripStaging_weather_df.count()

ShipName                   280280
ShipType                   161537
Nationality                280280
RecID                      280280
Year                       280280
Month                      280280
Day                        280280
Distance                   168970
ShipSpeed                      37
drLatDeg                   155413
Weather                     95390
WindForce                  249831
WindDirection              270238
PrecipitationDescriptor     24757
dtype: int64